# Day 3 - Performant Python

Python has a reputation for being efficient in terms of programmer calories, i.e., Python allows you to do a lot with little code and succinctly express your ideas, but ineficient in terms of clock cycles. However, it turns out that this isn't necessarily true. In fact, there are many ways to make Python go fast, some of which we'll cover today.

We'll start by talking about why Python is slow by default. Next, we'll talk about how to use numpy and numba to make it go fast. Over the course of the session you'll be given a few exercises.

## TODO:
* Profiling
* Nopython mode

## numpy
In most cases you should first try numpy if you need your code to go faster.
* numpy is fast because it's implemented in a manner optimized for numerical computing, whereas vanilla Python is designed for flexibility. For example, Python lists may contain any data types whereas numpy arrays have a fixed data type, e.g., float64.
* The difference is even more pronounced when you aren't using the built-in Python functions, in this case `sum()`, since the built-in functions often are more optimized than what is easy to write yourself.

In [ ]:
# built-in sum
import random
n = 100000

# vanilla Python sum
data = [random.random() for _ in range(n)] # n random floats
%timeit sum(data)

In [ ]:
# numpy sum
import numpy as np
data = np.random.random(n) # n random floats
%timeit data.sum()

In [ ]:
# Python diff
n = 100000
def diff(l):
    '''compute the derivative of l in-place.'''
    if not len(l):
        return l
    for i in range(1, len(l)):
        l[i] = l[i] - l[i-1]
    l[0] = 0
    return l

data = [random.random() for _ in range(n)] # n random floats
%timeit diff(data)

In [ ]:
# numpy diff
import numpy as np
data = np.random.random(n) # n random floats
%timeit np.diff(data)

### Exercise
* Show loop vs. array operations. For example with diff. Array operations faster.
* You're find as long as you can use numpy array operations, i.e., few individual array indexing.

## numba
As long as your needs are covered by the numpy and scipy libraries you don't need to worry about performance. This is great news since this will probably be true for most of what you do. But we need a way to make Python go fast also in cases where numpy isn't enough! One common strategy is to write your performance-critical functions in C/Fortran and call those functions from Python via a wrapper. But, there's a better way.

In this session, we're going to cover numba, a just-in-time compiler for Python. Essentially its a system that compiles Python code into machine code similar to what you would get from C. This means you can write Python code that runs almost as fast as C code.

* Compiling functions can bring big performance gains.
* Traditional languages like C are compiled ahead of time before the program can be started. Just-in-time compuling (jitting) is the process of compiling functions the first time a function is called during program execution.
* Jitting can bring the benefits of compilation that languages like C enjoy without sacrificing the convenience of Python.
* numba compiles optimized code for whatever data types your arguments are.

### JIT-ing Arbitrary Functions

In [ ]:
from numba import jit, njit
n = 100000

@njit
def diff(l):
    if not len(l):
        return l
    for i in range(1, len(l)):
        l[i] = l[i] - l[i-1]
    l[0] = 0
    return l

data = np.random.random(n) # n random floats
diff(data)
%timeit diff(data)

### Exercise: cumsum
Write a function `cumsum` that takes an array as its single argument and returns a new array of the same length, where the i-th element is the sum of all elements with indices smaller than or equal to i. Time it with an without the `@njit` decorator. How fast can you make it?

TODO: Show the line profiler before this assignment.

In [ ]:
@njit
def cumsum(a):
    '''return the cumulative sum of a'''
    rv = np.zeros(len(a))
    if not len(a):
        return rv
    rv[0] = a[0]
    for i in range(1, len(a)):
        rv[i] = rv[i-1] + a[i]
    return rv

# test the function
ans = cumsum(np.array([1, 2, 3]))
correct = np.array([1, 3, 6])
assert np.allclose(ans, correct), f'expected {correct}, but got {ans}'

ans = cumsum(np.array([1, -2, 3]))
correct = np.array([1, -1, 2])
assert np.allclose(ans, correct), f'expected {correct}, but got {ans}'

ans = cumsum(np.array([]))
correct = np.array([])
assert np.allclose(ans, correct), f'expected {correct}, but got {ans}'

print('all tests pass')

# time your code with and without the jit decorator
n = 10000
data = np.random.random(n)
sum_max_3(data) # warm up the jit
%timeit cumsum(data)

### Exercise: sum-max-3
Write a function `sum_max_3` that takes an array as its single argument and returns the sum of the three largest values in the array. Time it with an without the `@njit` decorator. How fast can you make it?

In [ ]:
import math

@njit
def sum_max_3(a):
    '''return the sum of the 3 largest values in a'''
    assert len(a) >= 3, 'input array must have at least 3 elements'
    max4 = np.zeros(4)
    max4[0] = -math.inf
    max4[1:] = a[:3]
    max4.sort()
    for v in a[3:]:
        max4[0] = v
        if max4[0] > max4[1]:
            max4[0], max4[1] = max4[1], max4[0]
            if max4[1] > max4[2]:
                max4[1], max4[2] = max4[2], max4[1]
                if max4[2] > max4[3]:
                    max4[2], max4[3] = max4[3], max4[2]
    return max4[1:].sum()

# test the function
ans = sum_max_3(np.array([1, 2, 3]))
correct = 6
assert ans == correct, f'expected {correct}, but got {ans}'

ans = sum_max_3(np.array([1, 1, 2, 3]))
correct = 6
assert ans == correct, f'expected {correct}, but got {ans}'

ans = sum_max_3(np.array([1, 2, 3, 10]))
correct = 15
assert ans == correct, f'expected {correct}, but got {ans}'

ans = sum_max_3(np.array([-1, 11, 30, -40, 10]))
correct = 51
assert ans == correct, f'expected {correct}, but got {ans}'

ans = sum_max_3(np.array([-1, -11, -30, -40, 10]))
correct = -2
assert ans == correct, f'expected {correct}, but got {ans}'

print('all tests pass')

# time your code with and without the jit decorator
n = 10000
data = np.random.random(n)
sum_max_3(data) # warm up the jit
%timeit sum_max_3(data)

## Numba and Objects
In many cases its good practice to write object oriented code. For example, if you have a program dealing with 2D vectors its often easier to create a class representing the vector and then write functions that take the object as its argument rather than passing around the vector coordinates explicitly. Let's look at an example.

TODO: Consider putting custom dtyper first. Or use tuple/namedtuple instead of objects. Or dictionaries.

In [ ]:
import math

class Vector2D:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    def __repr__(self):
        return f'Vector2D(x: {self.x}, y={self.y})'

def distance(v1, v2):
    '''return the distance between vectors v1 and v2'''
    return math.sqrt(math.pow(v1.x-v2.x, 2) + math.pow(v1.y-v2.y, 2))

def shortest_distance(vs):
    '''return the shortest distance between any pair of vectors.'''
    n = len(vs)
    mind = math.inf
    for i in range(n):
        for j in range(i+1, n):
            v1, v2 = vs[i], vs[j]
            # d = math.sqrt(math.pow(v1.x-v2.x, 2) + math.pow(v1.y-v2.y, 2))
            d = distance(v1, v2)
            mind = min(d, mind)
    return mind

# benchmark performance
n = 100 # number of vectors
vectors = [Vector2D(random.random(), random.random()) for _ in range(n)]
%timeit shortest_distance(vectors)

### Custom dtypes
Unfortunately numba doesn't support objects. However, we can use numpy custom data types (dtypes) to the same effect! For the C programmers, custom dtypes are essentially C structs, i.e., not full-fledged objects.

TODO: Emphasize that custom dtypes are for organizing code and then making it fast. Not just to make code fast.

In [ ]:
vector2d_dtype = np.dtype([
    ('x', np.float64),
    ('y', np.float64),
])

def create_vectors(xs, ys):
    '''create an array of vectors (xs[i], ys[i]) for all i'''
    return np.array([(x, y) for x, y in zip(xs, ys)], dtype=vector2d_dtype)

xs = [1, 2]
ys = [3, 4]
vs = create_vectors(xs, ys)

print(vs)

v = vs[0] # get the first point
print(v)
print(v['x'], v['y']) # access elements of the "object"

#### Let's solve the shortest distance problem using our custom vector dtype

In [ ]:
@njit # show that this is needed
def distance(v1, v2):
    '''return the distance between vectors v1 and v2'''
    return math.sqrt(math.pow(v1['x'] - v2['x'], 2) + math.pow(v1['y'] - v2['y'], 2))

@njit
def shortest_distance(vs):
    '''return the shortest distance between any pair of vectors'''
    n = len(vs)
    mind = math.inf
    for i in range(n):
        for j in range(i+1, n):
            v1, v2 = vs[i], vs[j]
            # d = math.sqrt(math.pow(v1['x']-v2['x'], 2) + math.pow(v1['y']-v2['y'], 2))
            d = distance(v1, v2)
            mind = min(d, mind)
    return mind

vectors = create_vectors(np.random.random(n), np.random.random(n))
shortest_distance(vectors) # warm up the jit
%timeit shortest_distance(vectors)

## Custom ufuncs
In numpy there are lots of functions that can take either scalar or vector arguments. These are referred to as ufuncs and they used to be quite complicated to make.

In [ ]:
from numba import vectorize

@vectorize
def divide_by_larger(a, b):
    '''return the smaller number of a and b divided by the larger number'''
    if a > b:
        return b / a
    return a / b

a = 2
b = 3
rv = divide_by_larger(a, b)
print(rv)

n = 100000
a = 2 * np.ones(n)
b = 3 * np.ones(n)
rv = divide_by_larger(a, b)
print(rv)

## Automagic Parallelization
Numba also allows us to write code that automatically executes in parallel.
* Automagically parallelize parallelize your code to exploit multi-core CPUs and even GPUs.
* https://numba.pydata.org/numba-doc/dev/user/vectorize.html#dynamic-universal-functions

In [ ]:
from numba import float64

@vectorize([float64(float64, float64)], target='parallel')
def divide_by_larger(a, b):
    '''return the smaller number of a and b divided by the larger number'''
    if a > b:
        return b / a
    return a / b

n = 100000
a = 2 * np.ones(n)
b = 3 * np.ones(n)
%timeit divide_by_larger(a, b)

# Exercises
* Give them a function written using loops that they should rewrite using array operations.
* Consider giving them a problem and a naive implementation and have them make it go fast. Should be something a bit complicated. Should be some speedup from using njit (or njit doens't work without changes) but there should be additional things to do as well. What's the speedup factor compared to the naive implementation?